In [1]:
import os
import json

# 공통 path 지정
rawPath = os.path.abspath(os.path.join(os.getcwd(), '1_RawData')) # raw 데이터 있는 곳
path = os.getcwd()

# 다른 노트북 코드 불러와서 실행
codeFileNames = ['setting', 'data_subset', 'vis_line_graph', 'Correlation', 'pair_log_rate','adf_pair']

for name in codeFileNames :
    print(name)
    
    k = json.load(open("./2_Modules/" + name + ".ipynb", 'r', encoding='utf-8'))
    all_code_lst =[]
    for cell in k['cells']:
        if cell['cell_type']=='code':
            all_code_lst.append("".join(cell['source']))
    all_code = "\n".join(all_code_lst)
    exec(all_code)


setting
data_subset
vis_line_graph
설명 : 주식 코드 데이터를 받아서, 선형 그래프를 그린 후 img 폴더에 저장       우측에 있는 종목이 보조축을 사용 매개변수 :raw_df = 상관관계를 보기 위한 데이터, 데이터프레임 형식         pair_df = 페어 데이터, 데이터프레임 형식         graph_name = 그래프의 주제 입력, 시각화 이미지가 저장될때 곂쳐지지 않기 위함                      폴더를 만들때 이 이름 기준으로 만드므로 조심하여 사용할 것                      ex) '가격_오름차순', '수익률_내림차순'         is_add_yaxis = 보조축 추가 여부, bool 형식         scr_col_num = 점수(상관계수 등)가 들어있는 컬럼 위치         strt.dt/ end.dt = 시작날짜/ 끝날짜, 문자열로 입력('2018-01-01'), 입력 없으면 전체 날짜         code_name = 주식 코드와 이름 매핑. 전처리 과정에서 만든 dic_code_name 사용 필수사항 : 데이터의 컬럼 1,2번은 주식 코드여야 함
Correlation
pair_log_rate
adf_pair


In [2]:
# 년도별로 allPrice, allRetrun, corPrice, corReturn의 df 다시 만든다
# corPrice, corReturn은 주가와 수익률에 대한 종목 간 상관관계 df

for i in range(2015, 2019, 1):
    start = str(i) + '-01-01'
    end = str(i) + '-12-31'
    globals()['allPrice_%s' % i] = get_data_by_date(allPrice,start,end)
    globals()['allReturn_%s' % i] = get_data_by_date(allReturn,start,end)
    globals()['corPrice_%s' % i] = corr_pair(globals()['allPrice_%s' % i], absolute = False)
    globals()['corReturn_%s' % i] = corr_pair(globals()['allReturn_%s' % i], absolute = True)
    
# 주가 상관관계 상위 10%, 수익률 상관관계 하위 10%
subPrice_2015 = sub_pair(corPrice_2015,ratio=0.1, down = False, absolute = False)
subReturn_2015 = sub_pair(corReturn_2015,ratio=0.1, down=True, absolute = True)

# inner join 할 key 생성
subPrice_2015["key"] = subPrice_2015["pair1"] + subPrice_2015["pair2"]
subReturn_2015["key"] = subReturn_2015["pair1"] + subReturn_2015["pair2"]

# inner join
merge_2015 = pd.merge(subPrice_2015,subReturn_2015,how = 'inner',on='key')

# 필요없는(중복) 컬럼 제거
merge_2015 = merge_2015.drop(['key','pair1_y','pair2_y'],axis=1)

# 컬럼명 변경
merge_2015 = merge_2015.rename(columns={'pair1_x':'pair1',
                                      'pair2_x':'pair2',
                                      'cor_x':'priceCor',
                                      'absCor_x':'priceAbsCor',
                                      'cor_y':'returnCor',
                                      'absCor_y':'returnAbsCor'})

pairLog = sub_pair_log_rate(merge_2015, allPrice_2015)

pairLog.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,,2015-01-02 00:00:00,2015-01-05 00:00:00,2015-01-06 00:00:00,2015-01-07 00:00:00,2015-01-08 00:00:00,2015-01-09 00:00:00,2015-01-12 00:00:00,2015-01-13 00:00:00,2015-01-14 00:00:00,2015-01-15 00:00:00,...,2015-12-16 00:00:00,2015-12-17 00:00:00,2015-12-18 00:00:00,2015-12-21 00:00:00,2015-12-22 00:00:00,2015-12-23 00:00:00,2015-12-24 00:00:00,2015-12-28 00:00:00,2015-12-29 00:00:00,2015-12-30 00:00:00
pair1,pair2,,,,,,,,,,,,,,,,,,,,,
A033780,A109070,2.759882,2.735317,2.585674,2.581025,2.527829,2.513112,2.542536,2.544965,2.499162,2.483634,...,1.475798,1.395469,1.301232,1.324601,1.444272,1.507475,1.600509,1.644123,1.468159,1.448801
A001520,A092200,1.733203,1.711536,1.697177,1.663735,1.658541,1.634649,1.578055,1.579521,1.551128,1.516711,...,1.227091,1.256241,1.296261,1.229919,1.190876,1.155293,1.186260,1.113105,1.092642,1.104611
A000370,A008930,1.088527,1.106871,1.134445,1.154850,1.177452,1.153005,1.160617,1.155822,1.169870,1.163194,...,2.685176,2.686507,2.727677,2.748232,2.736572,2.703195,2.675679,2.692578,2.756375,2.725803
A005800,A104700,2.924123,2.944439,2.937296,2.936274,2.956858,2.930633,2.877949,2.842621,2.839506,2.827397,...,2.899498,2.900422,2.886233,2.875286,2.883353,2.848344,2.855248,2.848582,2.918402,2.896397
A000370,A072710,3.157482,3.159461,3.208835,3.170997,3.188804,3.175569,3.189455,3.242563,3.229997,3.217249,...,2.938566,2.953009,2.973826,3.009997,3.004341,3.005520,3.004966,3.010648,3.001632,3.004341


In [3]:
adfPair=adf_pair(pairLog)
adfPair.head()

,,ADF,pValue,CV_1,CV_5,CV_10
pair1,pair2,,,,,
A001520,A092200,-3.05203,0.030313,-3.45711,-2.87331,-2.57304
A011700,A109070,-2.94268,0.0406052,-3.45711,-2.87331,-2.57304
A007310,A109070,-3.49444,0.00813115,-3.45711,-2.87331,-2.57304
A001820,A033250,-3.89716,0.00205583,-3.45711,-2.87331,-2.57304
A002710,A004020,-3.50763,0.00779317,-3.45711,-2.87331,-2.57304


In [4]:
adf_pair_line_plot(allPrice_2015, adfPair, '1차_0527', is_log = False)

전체 373개중 0번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0000_A001520_A092200.png
전체 373개중 1번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0001_A011700_A109070.png
전체 373개중 2번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0002_A007310_A109070.png
전체 373개중 3번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0003_A001820_A033250.png
전체 373개중 4번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0004_A002710_A004020.png


File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0041_A000060_A192400.png
전체 373개중 42번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0042_A002870_A044820.png
전체 373개중 43번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0043_A010580_A011420.png
전체 373개중 44번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0044_A004090_A192400.png
전체 373개중 45번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0045_A047050_A180640.png
전체 373개중 46번 실행중
phantomjs version: 2.1.1

Generating file

File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0082_A006090_A115390.png
전체 373개중 83번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0083_A008870_A104700.png
전체 373개중 84번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0084_A000140_A034730.png
전체 373개중 85번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0085_A001780_A044450.png
전체 373개중 86번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0086_A009180_A104700.png
전체 373개중 87번 실행중
phantomjs version: 2.1.1

Generating file

File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0123_A002820_A003690.png
전체 373개중 124번 실행중
phantomjs version: 2.1.1

Generating file ...
File saved in /Users/earllee1/googledrive_earllee87/study/분석프로젝트/PairTrading/9_vis_img/1차_0527_20150102_20151230/0124_A071840_A084690.png
전체 373개중 125번 실행중
phantomjs version: 2.1.1

Generating file ...


KeyboardInterrupt: 